# Community Centre Dataset

In [15]:
!pip install geopandas pyarrow beautifulsoup4

In [16]:
import geopandas as gpd
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Load dataset 

In [17]:
# Community Clubs GeoJSON file 
geojson_file = "CommunityClubs.geojson"
gdf = gpd.read_file(geojson_file)

# Show first few rows to inspect
display(gdf.head())

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.79224 1.42749 0)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.88955 1.35466 0)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.88667 1.39713 0)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.84067 1.36683 0)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.74754 1.3208 0)


# Extract columns

In [18]:
# Function to extract data from the 'Description' HTML field
def extract_description_info(description):
    soup = BeautifulSoup(description, "html.parser")
    data = {}

    # Extract the table rows from the HTML
    rows = soup.find_all('tr')

    # Loop through each row and extract the column names and values
    for row in rows:
        th_elements = row.find_all('th')
        td_elements = row.find_all('td')
        
        # Check if both <th> and <td> exist before extracting
        if th_elements and td_elements:
            key = th_elements[0].get_text(strip=True)
            value = td_elements[0].get_text(strip=True)
            data[key] = value

    return data

# Apply the extraction function to the 'Description' column
description_data = gdf['Description'].apply(extract_description_info)

# Convert the extracted data into a DataFrame
description_df = pd.json_normalize(description_data)

# Combine the new columns with the original GeoDataFrame (without overwriting the existing ones)
gdf = pd.concat([gdf, description_df], axis=1)

# Now, the GeoDataFrame should have new columns extracted from the Description field
print(gdf.head())


    Name                                        Description  \
0  kml_1  <center><table><tr><th colspan='2' align='cent...   
1  kml_2  <center><table><tr><th colspan='2' align='cent...   
2  kml_3  <center><table><tr><th colspan='2' align='cent...   
3  kml_4  <center><table><tr><th colspan='2' align='cent...   
4  kml_5  <center><table><tr><th colspan='2' align='cent...   

                        geometry ADDRESSBUILDINGNAME ADDRESSFLOORNUMBER  \
0  POINT Z (103.79224 1.42749 0)                                          
1  POINT Z (103.88955 1.35466 0)                                      1   
2  POINT Z (103.88667 1.39713 0)                                          
3  POINT Z (103.84067 1.36683 0)                                          
4   POINT Z (103.74754 1.3208 0)                                          

  ADDRESSPOSTALCODE    ADDRESSSTREETNAME ADDRESSTYPE ADDRESSUNITNUMBER  \
0            739069   Woodlands Avenue 1           A                     
1            530110   

In [19]:
# List all column names
print(gdf.columns.tolist())

['Name', 'Description', 'geometry', 'ADDRESSBUILDINGNAME', 'ADDRESSFLOORNUMBER', 'ADDRESSPOSTALCODE', 'ADDRESSSTREETNAME', 'ADDRESSTYPE', 'ADDRESSUNITNUMBER', 'DESCRIPTION', 'HYPERLINK', 'LANDXADDRESSPOINT', 'LANDYADDRESSPOINT', 'PHOTOURL', 'ADDRESSBLOCKHOUSENUMBER', 'NAME', 'INC_CRC', 'FMEL_UPD_D']


# Dataset statistics

In [20]:
# Get the number of rows
num_rows = gdf.shape[0]
print(f"Number of rows in the dataset: {num_rows}")

Number of rows in the dataset: 125


In [21]:
# Check for empty strings in the entire GeoDataFrame -- same as checking for null values 
empty_strings = (gdf == "").sum()

# Display columns with empty strings
print("Empty strings per column:")
print(empty_strings)

Empty strings per column:
Name                         0
Description                  0
geometry                     0
ADDRESSBUILDINGNAME        120
ADDRESSFLOORNUMBER         104
ADDRESSPOSTALCODE            0
ADDRESSSTREETNAME            0
ADDRESSTYPE                 11
ADDRESSUNITNUMBER          104
DESCRIPTION                  0
HYPERLINK                   19
LANDXADDRESSPOINT          125
LANDYADDRESSPOINT          125
PHOTOURL                   125
ADDRESSBLOCKHOUSENUMBER      0
NAME                         0
INC_CRC                      0
FMEL_UPD_D                   0
dtype: int64


In [22]:
# Check for duplicates 

# CC NAME
duplicates = gdf.duplicated(subset=['NAME']).sum()
print(f"Number of duplicate CC names: {duplicates}")

# Count the number of rows with DESCRIPTION = 'CC'
cc_rows = gdf[gdf['DESCRIPTION'] == 'CC'].shape[0]
print(f"Number of rows with DESCRIPTION = 'CC': {cc_rows}")

# Count the number of rows with DESCRIPTION != 'CC'
non_cc_rows = gdf[gdf['DESCRIPTION'] != 'CC'].shape[0]
print(f"Number of rows with DESCRIPTION != 'CC': {non_cc_rows}")


Number of duplicate CC names: 0
Number of rows with DESCRIPTION = 'CC': 119
Number of rows with DESCRIPTION != 'CC': 6


In [23]:
# Filter rows where DESCRIPTION is not "CC"
filtered_description = gdf[gdf["DESCRIPTION"] != "CC"]
print("Rows where DESCRIPTION is not 'CC':")
display(filtered_description)

Rows where DESCRIPTION is not 'CC':


,Name,Description,geometry,ADDRESSBUILDINGNAME,ADDRESSFLOORNUMBER,ADDRESSPOSTALCODE,ADDRESSSTREETNAME,ADDRESSTYPE,ADDRESSUNITNUMBER,DESCRIPTION,HYPERLINK,LANDXADDRESSPOINT,LANDYADDRESSPOINT,PHOTOURL,ADDRESSBLOCKHOUSENUMBER,NAME,INC_CRC,FMEL_UPD_D
31,kml_32,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.86678 1.29499 0),,,437440,Rhu Cross,A,,PAssion WaVe,,,,,11,PAssion WaVe @ Marina Bay (Sea Sports Club),549443D102A6D8DF,20211007162617
32,kml_33,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.94345 1.38495 0),,,519926,Elias Road,A,,PAssion WaVe,,,,,125,PAssion WaVe @ Pasir Ris (Sea Sports Club),3A5DB08F27CADF32,20211007162617
33,kml_34,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.84365 1.45904 0),,,759058,Jalan Mempurong,A,,PAssion WaVe,,,,,60,PAssion WaVe @ Sembawang (Sea Sport Club),C584161F2CD43E8D,20211007162617
122,kml_123,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.93301 1.34126 0),,,479311,Bedok Reservoir Road,A,,PAssion WaVe,,,,,911,PAssion WaVe @ Bedok Reservoir (Sea Sports Clu...,9D49A913C46D0E69,20211109134550
123,kml_124,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.95222 1.31173 0),,,468961,East Coast Parkway,A,,PAssion WaVe,,,,,1390,PAssion WaVe @ East Coast (Sea Sports Club),F19BA33D757F9C7B,20211007162617
124,kml_125,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.7273 1.33936 0),,,618662,Yuan Ching Road,A,,PAssion WaVe,,,,,100,PAssion WaVe @ Jurong Lake Gardens (Sea Sports...,FF0B51F54EEFA927,20211007162617


# Clean the dataset
1. convert geometry into lat, lon form 
2. drop columns 
3. rename and rearrange columns 

In [24]:
# Extract longitude and latitude from the geometry column
gdf['longitude'] = gdf['geometry'].apply(lambda point: point.x)  # Longitude is the x-coordinate
gdf['latitude'] = gdf['geometry'].apply(lambda point: point.y)   # Latitude is the y-coordinate

# Verify the new columns 
print(gdf.head())

    Name                                        Description  \
0  kml_1  <center><table><tr><th colspan='2' align='cent...   
1  kml_2  <center><table><tr><th colspan='2' align='cent...   
2  kml_3  <center><table><tr><th colspan='2' align='cent...   
3  kml_4  <center><table><tr><th colspan='2' align='cent...   
4  kml_5  <center><table><tr><th colspan='2' align='cent...   

                        geometry ADDRESSBUILDINGNAME ADDRESSFLOORNUMBER  \
0  POINT Z (103.79224 1.42749 0)                                          
1  POINT Z (103.88955 1.35466 0)                                      1   
2  POINT Z (103.88667 1.39713 0)                                          
3  POINT Z (103.84067 1.36683 0)                                          
4   POINT Z (103.74754 1.3208 0)                                          

  ADDRESSPOSTALCODE    ADDRESSSTREETNAME ADDRESSTYPE ADDRESSUNITNUMBER  \
0            739069   Woodlands Avenue 1           A                     
1            530110   

DROP / KEEP : (read in edit mode)

- Name                         0 (because its kml_1, kml_2, kml_3 ...)
- Description                  0 (because this is the original html table that stores the capital letter columns)
- geometry                     0 
- ADDRESSBUILDINGNAME        120 (mostly are null)
- ADDRESSFLOORNUMBER         104 (mostly are null)
- ADDRESSPOSTALCODE            0 
- ADDRESSSTREETNAME            0
- ADDRESSTYPE                 11 (no explanation on this so drop -- either "A" or null)
- ADDRESSUNITNUMBER          104 (mostly are null)
- DESCRIPTION                  0 
- HYPERLINK                   19 (not needed as the name suggest)
- LANDXADDRESSPOINT          125 (all are null)
- LANDYADDRESSPOINT          125 (all are null)
- PHOTOURL                   125 (all are null)
- ADDRESSBLOCKHOUSENUMBER      0 
- NAME                         0 
- INC_CRC                      0 (no explanation on this so drop)
- FMEL_UPD_D                   0 (no explanation on this so drop)

In [25]:
# Drop the unnecessary columns 
columns_to_drop = ['Name', 'Description', 'geometry', 'ADDRESSBUILDINGNAME', 'ADDRESSFLOORNUMBER', 'ADDRESSTYPE', 'ADDRESSUNITNUMBER', 'HYPERLINK', 'LANDXADDRESSPOINT', 'LANDYADDRESSPOINT', 'PHOTOURL', 'INC_CRC', 'FMEL_UPD_D']
gdf = gdf.drop(columns=columns_to_drop)

# Verify that the columns are dropped
print(gdf.head())

  ADDRESSPOSTALCODE    ADDRESSSTREETNAME DESCRIPTION ADDRESSBLOCKHOUSENUMBER  \
0            739069   Woodlands Avenue 1          CC                     120   
1            530110     Hougang Avenue 1          CC                     110   
2            544965      Anchorvale Road          CC                      59   
3            569976  Ang Mo Kio Avenue 1          CC                     795   
4            609335       Pandan Gardens          CC                     150   

               NAME   longitude  latitude  
0  ACE The Place CC  103.792241  1.427493  
1       Aljunied CC  103.889546  1.354658  
2     Anchorvale CC  103.886666  1.397128  
3     Ang Mo Kio CC  103.840668  1.366833  
4     Ayer Rajah CC  103.747540  1.320799  


In [ ]:
# Rename columns
gdf = gdf.rename(columns={
    'ADDRESSBLOCKHOUSENUMBER': 'blockNumber',
    'NAME': 'name',
    'ADDRESSSTREETNAME': 'address',
    'ADDRESSPOSTALCODE': 'postalCode',
    'DESCRIPTION' : 'CC / PassionWave'
})

# Rearrange columns
gdf = gdf[['name', 'address', 'postalCode', 'blockNumber', 'CC / PassionWave', 'longitude', 'latitude']]

# Sort the GeoDataFrame by hawker_centre_name alphabetically
gdf = gdf.sort_values(by='name', ascending=True)

# Verify the column names after renaming
print(gdf.head())

               name          street_name postal_code block_number  \
0  ACE The Place CC   Woodlands Avenue 1      739069          120   
1       Aljunied CC     Hougang Avenue 1      530110          110   
2     Anchorvale CC      Anchorvale Road      544965           59   
3     Ang Mo Kio CC  Ang Mo Kio Avenue 1      569976          795   
4     Ayer Rajah CC       Pandan Gardens      609335          150   

  CC / PassionWave   longitude  latitude  
0               CC  103.792241  1.427493  
1               CC  103.889546  1.354658  
2               CC  103.886666  1.397128  
3               CC  103.840668  1.366833  
4               CC  103.747540  1.320799  


# Save as CSV

In [27]:
csv_file = "community_clubs_cleaned.csv"
gdf.to_csv(csv_file, index=False)

# Load and preview CSV file to confirm
df = pd.read_csv(csv_file)
display(df.head())

,name,street_name,postal_code,block_number,CC / PassionWave,longitude,latitude
0,ACE The Place CC,Woodlands Avenue 1,739069,120,CC,103.792241,1.427493
1,Aljunied CC,Hougang Avenue 1,530110,110,CC,103.889546,1.354658
2,Anchorvale CC,Anchorvale Road,544965,59,CC,103.886666,1.397128
3,Ang Mo Kio CC,Ang Mo Kio Avenue 1,569976,795,CC,103.840668,1.366833
4,Ayer Rajah CC,Pandan Gardens,609335,150,CC,103.747540,1.320799
